In [1]:
# Example notebook structure
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, log_loss
import time


# Load data



In [2]:
X = np.load('cifar10_features.npy')
y = np.load('cifar10_labels.npy')



# Split data




In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Softmax Logistic Regression


In [4]:
beginning_time = time.time()
softmax = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
softmax.fit(X_train, y_train)
softmax_time = time.time() - beginning_time

softmax_predictions = softmax.predict(X_test)
softmax_accuracy = accuracy_score(y_test, softmax_predictions)
softmax_f1 = f1_score(y_test, softmax_predictions, average='weighted')
softmax_log_loss = log_loss(y_test, softmax.predict_proba(X_test))



c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



# One-vs-all Logistic Regression



In [5]:
beginning_time = time.time()
oneVsAll = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=5000)
oneVsAll.fit(X_train, y_train)
oneVsAll_time = time.time() - beginning_time

oneVsAll_predictions = oneVsAll.predict(X_test)
oneVsAll_accuracy = accuracy_score(y_test, oneVsAll_predictions)
oneVsAll_f1 = f1_score(y_test, oneVsAll_predictions, average='weighted')
oneVsAll_log_loss = log_loss(y_test, oneVsAll.predict_proba(X_test))


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


# Print results


In [6]:
print(f"Softmax Accuracy: {softmax_accuracy}, Time: {softmax_time} seconds, Log Loss: {softmax_log_loss}, F1 Score: {softmax_f1}")
print(f"One-vs-All Accuracy: {oneVsAll_accuracy}, Time: {oneVsAll_time} seconds, Log Loss: {oneVsAll_log_loss}, F1 Score: {oneVsAll_f1}")


Softmax Accuracy: 0.963, Time: 2.3493075370788574 seconds, Log Loss: 0.10771339288105258, F1 Score: 0.9630302185229656
One-vs-All Accuracy: 0.961, Time: 1.1832456588745117 seconds, Log Loss: 0.1345972383692102, F1 Score: 0.9610172357351848



# Confusion Matrix for one vs all



In [7]:
oneVsAllMatrix = confusion_matrix(y_test, oneVsAll_predictions)
print(oneVsAllMatrix)

[[1402    2   13    5    5    3    2   10   17    5]
 [   2 1464    4    1    0    1    0    1    5    7]
 [  11    1 1370   15   16   15    5    3    1    3]
 [  14    5   15 1450   12   51    5   10    3    4]
 [   6    0   13   15 1466    5    4    9    0    1]
 [   2    3   13   45   12 1437    8   11    2    1]
 [   5    5    9   17    4    6 1415    0    1    1]
 [   3    0    4   17    9    7    0 1456    0    1]
 [  13    2    1    5    1    0    2    0 1480    6]
 [  12    7    5    6    0    4    2    3    5 1475]]



# Train a specialized model for the hardest-to-distinguish class pair


In [8]:
class_pair = (3, 5)
class_pair_indices = np.where((y_train == class_pair[0]) | (y_train == class_pair[1]))

X_train_pair = X_train[class_pair_indices]
y_train_pair = y_train[class_pair_indices]

X_test_pair = X_test[np.where((y_test == class_pair[0]) | (y_test == class_pair[1]))]
y_test_pair = y_test[np.where((y_test == class_pair[0]) | (y_test == class_pair[1]))]

specialized_model = LogisticRegression(solver='lbfgs', max_iter=5000)
specialized_model.fit(X_train_pair, y_train_pair)

pair_predictions = specialized_model.predict(X_test_pair)
pair_accuracy = accuracy_score(y_test_pair, pair_predictions)
pair_f1 = f1_score(y_test_pair, pair_predictions, average='weighted')

print(f"Specialized Model Accuracy: {pair_accuracy}")
print(f"Specialized Model F1 Score: {pair_f1}")

Specialized Model Accuracy: 0.9622945536577505
Specialized Model F1 Score: 0.9622953212524546
